# Pre-Processing of the SEEG Signal
This notebook presents the pre-processing stages the SEEG signal goes through before being fed to the SNN. The pre-processing stages are as follows:
1. **Filtering**: The SEEG signal is bandpass filtered to remove noise and artifacts. The bandpass filter is designed using the Butterworth filter and, since we are working with *iEEG*, the signal is filtered in the ripples and FR bands. The co-occurrence of HFOs in both bands is an optimal prediction of post-surgical seizure freedom by defining an optimal "HFO area" or EZ zone.
2. **Signal-to-Spike Conversion**: To interface and communicate with the silicon neurons in the SNN, the SEEG signal must be converted to spikes.

## Filtering
Depending on the EEG modality, the signal is filtered in different frequency bands. In this case, since we are handling *iEEG* or *sEEG* data, the signal is filtered in both the ripples (80-250Hz) and FR bands (250-500Hz). The co-occurrence of HFO in these bands represents an optimal prediction of post-surgical seizure freedom by defining an optimal "HFO area" or EZ zone.

The filter is implemented in different ways depending on the setup it will run on.
1. **Neuromorphic Hardware**: The filter is implemented using analog filters. 
2. **Software Simulation**: *Butterworth filters* are utilized since they are a good approximation of the tuned *Tow-Thomas* architectures implemented in hardware.

The frequency response of the *Butterworth filter* is maximally flat in the passband and rolls of towards 0 in the stopband.

### Check WD and file loading

In [97]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

PATH_TO_FILE = '' # src/hfo/' # ''  # This is needed if the WD is not the same as the file location

/home/monkin/Desktop/feup/thesis/lava/src/hfo


In [98]:
import numpy as np
import math

seeg_file_name = "seeg_synthetic_humans.npy"
recorded_data = np.load(f"{PATH_TO_FILE}data/{seeg_file_name}")

print("Data shape: ", recorded_data.shape)
print("First time steps: ", recorded_data[:10])

Data shape:  (245760, 960)
First time steps:  [[ 3.2352024e-01 -1.3235390e+00 -5.9668809e-01 ... -1.9608999e+00
  -1.9769822e-01 -1.2078454e+00]
 [-6.9759099e-04 -3.5122361e+00 -4.8766956e-01 ... -5.8757830e+00
  -7.4400985e-01 -5.1096064e-01]
 [ 1.9026639e+00 -5.6726017e+00  9.8274893e-01 ... -6.6182971e+00
  -8.3053267e-01 -8.1596655e-01]
 ...
 [ 3.2172418e+00 -8.4650068e+00  1.5216088e+00 ... -4.1081657e+00
   2.0085973e-01 -4.7539668e+00]
 [ 1.7725919e+00 -9.4744024e+00  1.6776791e+00 ... -4.1469693e+00
   1.6412770e+00 -3.4672713e+00]
 [ 7.8109097e-01 -1.0500931e+01  2.3717029e+00 ... -5.1762242e+00
   1.0715837e+00 -4.4489903e+00]]


### Add the parent directory to the path to detect the utils module

In [99]:
import os
import sys

# Add the parent directory to the path so it detects the utils module
module_path = os.path.abspath(os.path.join('src'))      # Changed this since WD is not the same as the file location
if module_path not in sys.path:
    sys.path.append(module_path)

## Define the Filter

In [100]:
from scipy.signal import butter, lfilter

# ================================================================ #
# ============ Butterworth Filter Coefficients =================== #
# ================================================================ #
def butter_bandpass(lowcut, highcut, sampling_freq, order=5):
    """
    This function is used to generate the coefficients for lowpass, highpass and bandpass
    filtering for Butterworth filters.
    @lowcut, highcut (int): cutoff frequencies for the bandpass filter
    @sampling_freq (float): sampling_frequency frequency of the wideband signal
    @order (int): filter order

    - return b, a (float): filtering coefficients that will be applied on the wideband signal
    """
    nyq = 0.5 * sampling_freq   # Nyquist frequency
    low = lowcut / nyq          # Normalizing the cutoff frequencies
    high = highcut / nyq        # Normalizing the cutoff frequencies

    return butter(order, [low, high], btype='band')    

# ================================================================ #
# ====================== Butterworth Filters ===================== #
# ================================================================ #
def butter_bandpass_filter(data, lowcut, highcut, sampling_freq, order=5):
    """
    This function applies the filtering coefficients calculated above to the wideband signal (original signal).
    @data (array): Array with the amplitude values of the wideband signal.
    @lowcut, highcut (int): cutoff frequencies for the bandpass filter.
    @sampling_freq (float): sampling frequency of the original signal.
    @order (int): filter order.

    - return (array): Array with the amplitude values of the filtered signal.
    """
    coef_b, coef_a = butter_bandpass(lowcut, highcut, sampling_freq, order)

    return lfilter(coef_b, coef_a, data)
    

### Extract a single SEEG channel from the SEEG data

In [101]:
selected_ch_idx = 63 # 1 32 63 94
seeg_ch = list(map(lambda all_channels: all_channels[selected_ch_idx], recorded_data))  # Selecting the first channel
seeg_ch_np = np.array(seeg_ch)

print(f"seeg_ch_np shape: {seeg_ch_np.shape}. \nPreview: {seeg_ch_np}")

seeg_ch_np shape: (245760,). 
Preview: [  0.23408996   0.31883836   2.7478404  ... 110.786125   111.722855
 112.302986  ]


## Define Global Parameters of the Experiment

In [102]:
sampling_rate = 2048    # 2048 Hz
input_duration = 120 * (10**3)    # 120000 ms or 120 seconds
num_samples = recorded_data.shape[0]    # 2048 * 120 = 245760
num_channels = recorded_data.shape[1]   # 960

x_step = 1/sampling_rate * (10**3)  # 0.48828125 ms

## Apply the Butterworth filter to the channel

In [103]:
# Filter the raw signal in the Ripple band (80-250 Hz)
ripple_band_seeg = butter_bandpass_filter(seeg_ch_np, 80, 250, sampling_rate, 5)

# Filter the raw signal in the Fast Ripple band (250-500 Hz)
fr_band_seeg = butter_bandpass_filter(seeg_ch_np, 250, 500, sampling_rate, 5)

## Import the Markers (Annotated Events) 
The markers are stored in a numpy array of shape (num_channels, events):
- Each row represents the events of a channel
- Each event is composed of the following 3 fields (Label, Position, Shape)

In [104]:
markers_seeg_file_name = "seeg_synthetic_humans_markers.npy"
markers = np.load(f"{PATH_TO_FILE}data/{markers_seeg_file_name}")

print("Markers shape: ", markers.shape)
print("First time steps: ", markers[:10])

Markers shape:  (960, 42)
First time steps:  [[('Spike+Ripple+Fast-Ripple',   1000.  , 0.)
  ('Spike+Ripple+Fast-Ripple',   4537.6 , 0.)
  ('Ripple+Fast-Ripple',   7610.84, 0.) ('Spike',  10261.2 , 0.)
  ('Fast-Ripple',  13411.6 , 0.) ('Spike+Ripple',  15644.  , 0.)
  ('Ripple',  18607.9 , 0.) ('Fast-Ripple',  21300.8 , 0.)
  ('Fast-Ripple',  24086.9 , 0.)
  ('Spike+Ripple+Fast-Ripple',  26598.1 , 0.)
  ('Spike+Fast-Ripple',  30529.8 , 0.) ('Ripple',  33387.7 , 0.)
  ('Spike+Fast-Ripple',  36145.5 , 0.) ('Spike',  38744.1 , 0.)
  ('Spike+Fast-Ripple',  41876.  , 0.)
  ('Ripple+Fast-Ripple',  44531.7 , 0.) ('Spike+Ripple',  47699.7 , 0.)
  ('Spike+Ripple+Fast-Ripple',  50303.2 , 0.) ('Ripple',  53045.9 , 0.)
  ('Ripple+Fast-Ripple',  56416.  , 0.)
  ('Spike+Ripple+Fast-Ripple',  58838.4 , 0.)
  ('Ripple+Fast-Ripple',  62578.6 , 0.) ('Fast-Ripple',  65430.2 , 0.)
  ('Spike',  68184.1 , 0.) ('Spike+Ripple',  70753.9 , 0.)
  ('Ripple+Fast-Ripple',  73788.6 , 0.) ('Spike',  76749.  , 0.)
  

## Visualize the filtered signals

In [105]:
# Interactive Plot for the HFO detection
# bokeh docs: https://docs.bokeh.org/en/2.4.1/docs/first_steps/first_steps_1.html

from utils.line_plot import create_fig  # Import the function to create the figure
from bokeh.models import Range1d

# Define the x and y values
# Should the first input start at 0 or x_step?
# TODO: is it okay to create a range with floats?
x = [val for val in np.arange(x_step, input_duration + x_step, x_step)] 

# Create the y arrays for the voltage plot representing the voltage of each electrode
v_yarrays = [ripple_band_seeg, fr_band_seeg]

## Create the Plot

In [106]:
# Create the plot
# List of tuples containing the y values and the legend label
hfo_y_arrays = [(ripple_band_seeg, "Ripple Band"), (fr_band_seeg, "Fast Ripple Band")]

# Create the SEEG Voltage plot
hfo_plot = create_fig(
    title="SEEG Voltage dynamics of Filtered Ripple and Fast Ripple Bands", 
    x_axis_label='time (ms)', 
    y_axis_label='Voltage (μV)',
    x=x, 
    y_arrays=hfo_y_arrays, 
    sizing_mode="stretch_both", 
    tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
    tooltips="Data point @x: @y",
    legend_location="top_right",
    legend_bg_fill_color="navy",
    legend_bg_fill_alpha=0.1,
    # y_range=Range1d(-0.05, 1.05)
)

# If there are more than 30 channels, hide the legend
if len(hfo_y_arrays) > 30:
    # Hide the legend
    hfo_plot.legend.visible = False

## Add Box Annotations to the plot to identify the marked HFOs (ground truth)

In [107]:
from bokeh.models import BoxAnnotation

show_markers = True    # Boolean to show the markers

confidence_range = 100          # TODO: Check this value. When the duration is missing (0), we consider the 200ms window around the marked position 
color_map = {                   # Map the label to a color
    'Spike': 'red',
    'Fast-Ripple': 'blue',
    'Ripple': 'green',  
    'Spike+Ripple': 'yellow',
    'Spike+Fast-Ripple': 'pink',
    'Ripple+Fast-Ripple': 'cyan',
    'Spike+Ripple+Fast-Ripple': 'black'
}
visited_markers = {}    # Avoid inserting multiple boxes for the same marker (only one of each label)
use_visited = False     # Boolean controlling if we remove duplicate markers
plot_instant = True     # Boolean to plot the markers as instant events or as boxes
instant_width = 100 # 20       # Width of the instant event for visualization purposes

channels_used = {selected_ch_idx}   # Set of channels to be used
if show_markers:
    for ch_idx in channels_used:
        channel_markers = markers[ch_idx]
        # print("channel_markers", channel_markers)
        for idx2, marker in enumerate(channel_markers):
            # print("marker:", marker)
            
            if use_visited:
                # Check if the marker has already been visited and skip it if it has
                if marker['position'] in visited_markers:
                    visited_labels = visited_markers[marker['position']]    # Get the labels that already have an annotation for this position
                    if marker['label'] in visited_labels:
                        # print("Skipping marker", marker['position'], marker['label'])
                        continue    # Skip this marker
                    else:
                        visited_labels.append(marker['label'])  # Add the label to the visited labels
                else:
                    visited_markers[marker['position']] = [marker['label']] # Add the marker to the visited markers

            # Add a box annotation for each marker
            has_duration = marker['duration'] > 0
            
            confidence_constant = 0 if plot_instant or has_duration else confidence_range

            left = marker['position'] - confidence_constant
            right = marker['position'] + confidence_constant + instant_width
            box_color = color_map[marker['label']]  # Choose a color according to the label
            
            # if left < min_t or right > max_t:
            #     continue    # Skip this marker
            

            box = BoxAnnotation(left=left, right=right, fill_color=box_color, fill_alpha=0.35)
            # print("Added marker for channel: ", ch_idx, " at position: ", left)
            hfo_plot.add_layout(box)

## Show the Plot

In [108]:
import bokeh.plotting as bplt

showPlot = True
if showPlot:
    bplt.show(hfo_plot)

## Export the plot to a file

In [109]:
export = False

if export:
    file_path = f"{PATH_TO_FILE}results/filtered_seeg_ch63.html"

    # Customize the output file settings
    bplt.output_file(filename=file_path, title="SEEG Data - Filtered Voltage dynamics across time")

    # Save the plot
    bplt.save(hfo_plot)